# Testing optimization by random search

In [1]:
import numpy

In [2]:
import sklearn

In [3]:
import survhive

In [4]:
seed=2402
n_cpus=4

In [5]:
X, y = survhive.load_test_data()
X.shape, y.shape

((198, 84), (198,))

### Generate a (stratified) train-test split and Scale the features (only) 

First do the stratified splitting THEN do scaling, parameterized on X_train set ONLY 

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
X_train, X_test, y_train, y_test = survhive.survival_train_test_split(X, y,rng_seed=seed)

In [8]:
scaler = StandardScaler().fit(X_train)
[X_train, X_test] = [ scaler.transform(_) for _ in  [X_train, X_test] ]
X_train.shape, X_test.shape

((148, 84), (50, 84))

In [9]:
#X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

balanced partitioning OK. Robst scaler damages the performance of DSM A LOT.
maybe did something wrong. It is standard scaler for now.

In [10]:
survhive.get_indicator(y).sum(), survhive.get_indicator(y_train).sum(), survhive.get_indicator(y_test).sum(),


(51, 38, 13)

In [11]:
splitter = survhive.survival_crossval_splitter(X_train,y_train,n_splits=3, n_repeats=2,rng_seed=2309)
print([ (survhive.get_indicator(y_train[_[1]]).sum()) for _ in splitter])

[13, 12, 13, 13, 12, 13]


## check possible dimensionality reduction

In [12]:
from sklearn.decomposition import PCA

In [13]:
pca= PCA(n_components=0.995, random_state=seed).fit(X_train)
pca.n_components_

74

Only a modest dimensionality reduction is possible using PCA

In [14]:
## Stratified CV spliter for survival analysis

In [15]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

# test coxnet

In [16]:
coxnet = survhive.CoxNet(rng_seed=seed)
coxnet.fit(X_train, y_train)

CoxNet(rng_seed=2402, alpha=None, l1_ratio=0.5)

In [17]:
coxnet.score(X_test, y_test)

0.5

In [18]:
#from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [19]:
survhive.optimization._guess_tries(survhive.CoxNet.get_parameter_grid())

6

In [18]:
survhive.CoxNet().get_parameter_grid()

{'alpha': [0.001,
  0.003,
  0.005,
  0.008,
  0.01,
  0.02,
  0.03,
  0.04,
  0.05,
  0.06,
  0.07,
  0.08,
  0.09,
  0.1,
  0.15,
  0.2,
  0.3],
 'l1_ratio': [0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99]}

In [20]:
from survhive import optimize

In [21]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = optimize(survhive.CoxNet(rng_seed=seed), X_train, y_train, 
                                                            mode='sklearn-random',
                                                           n_jobs=n_cpus)
opt_coxnet.score(X_test, y_test), opt_coxnet_params

Random search tries: 6


/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/ivan/Unito/survhive/survhive/sksurv_adapters.py", line 63, in fit
    self.model_ = self.model_.fit(X, y)
  File "/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/linear_model/coxnet.py", line 269, in fit
    coef, alphas

(0.572429906542056, {'l1_ratio': 0.75, 'alpha': 0.03})

In [23]:
opt_coxnet, opt_coxnet_params, opt_coxnet_search = optimize(survhive.CoxNet(rng_seed=seed), X_train, y_train,
                                                           n_jobs=n_cpus)
opt_coxnet.score(X_test, y_test), opt_coxnet_params

/usr/local/ivan/Unito/survhive/survhive/sksurv_adapters.py:63: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  self.model_ = self.model_.fit(X, y)
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/linear_model/coxph.py:64: RuntimeWarning: overflow encountered in exp
  risk_score = np.exp(linear_predictor)
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/linear_model/coxph.py:75: RuntimeWarning: invalid value encountered in scalar subtract
  value -= risk_score[k : (k + d)].sum()
/usr/local/ivan/Unito/conda/envs/hive/lib/python3.8/site-packages/sksurv/linear_model/coxph.py:120: RuntimeWarning: overflow encountered in exp
  risk_score = np.exp(linear_predictor)
/usr/local/ivan/Unito/survhive/survhive/sksurv_adapters.py:63: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  self.model_ = self.mod

(0.5677570093457944, {'alpha': 0.02, 'l1_ratio': 0.75})

## Test GrBSA

In [22]:
grbsa=survhive.GrBoostSA(rng_seed=seed)
grbsa

GrBoostSA(rng_seed=2402, n_estimators=100, max_depth=None, min_samples_split=0.1, min_samples_leaf=0.05, validation_fraction=0.1, patience=5)

In [23]:
grbsa.fit(X_train,y_train)
grbsa.score(X_test,y_test)

0.49065420560747663

In [24]:
grbsa.model_.n_estimators_

19

In [25]:
grbsa_grid = grbsa.get_parameter_grid()
grbsa_grid['n_estimators']=[100]
grbsa_grid['max_depth']=[None]
#grbsa_grid['validation_fraction']=[0.1,0.3,0.2]
grbsa_grid['min_samples_split']=[0.1,0.05]
grbsa_grid['min_samples_leaf']=[0.05, 0.02, 0.01]
#grbsa_grid['patience']=[5]
grbsa_grid

{'n_estimators': [100],
 'max_depth': [None],
 'min_samples_split': [0.1, 0.05],
 'min_samples_leaf': [0.05, 0.02, 0.01],
 'patience': [None, 5]}

In [26]:
scorers=survhive.metrics._SCORERS
scorers

{'c-index-antolini': <function survhive.metrics.concordance_index_antolini_scorer(estimator, X, y, return_all=False)>,
 'c-index-quartiles': <function survhive.metrics.make_survival_scorer.<locals>.scorer(estimator, X, y)>,
 'roc-auc-quartiles': <function survhive.metrics.make_survival_scorer.<locals>.scorer(estimator, X, y)>,
 'neg-brier-quartiles': <function survhive.metrics.make_survival_scorer.<locals>.scorer(estimator, X, y)>}

In [27]:
opt_rsf, opt_rsf_params, opt_rsf_search = optimize(survhive.GrBoostSA(rng_seed=seed), X_train, y_train, 
                                                   user_grid=grbsa_grid, scoring=scorers, 
                                                   refit='c-index-antolini',
                                                   #refit='neg-brier-quartiles',
                                                   n_jobs=n_cpus)
#opt_rsf, opt_rsf_params, opt_rsf_search = optimize(survhive.RSF(rng_seed=2309), X_train, y_train, n_jobs=4)
opt_rsf.score(X_train, y_train), opt_rsf.score(X_test, y_test), opt_rsf_params

(0.9958945548833189,
 0.6845794392523364,
 {'max_depth': None,
  'min_samples_leaf': 0.01,
  'min_samples_split': 0.1,
  'n_estimators': 100,
  'patience': None})

In [28]:
survhive.get_model_scores_df(opt_rsf_search)

,rank_test_c-index-antolini,mean_test_c-index-antolini,std_test_c-index-antolini,rank_test_c-index-quartiles,mean_test_c-index-quartiles,std_test_c-index-quartiles,rank_test_roc-auc-quartiles,mean_test_roc-auc-quartiles,std_test_roc-auc-quartiles,rank_test_neg-brier-quartiles,mean_test_neg-brier-quartiles,std_test_neg-brier-quartiles,params,mean_fit_time,std_fit_time
8,1,0.677028,0.082136,2,0.66808,0.080758,1,0.699151,0.144763,9,-0.11809,0.018454,"{'max_depth': None, 'min_samples_leaf': 0.01, ...",0.954163,0.12268
7,2,0.676703,0.069577,1,0.672184,0.07159,2,0.69882,0.128825,3,-0.113248,0.00911,"{'max_depth': None, 'min_samples_leaf': 0.02, ...",0.090216,0.047694
0,3,0.671129,0.09189,3,0.658716,0.090521,3,0.693279,0.134353,1,-0.112672,0.012165,"{'max_depth': None, 'min_samples_leaf': 0.05, ...",0.592562,0.055772
2,3,0.671129,0.09189,3,0.658716,0.090521,3,0.693279,0.134353,1,-0.112672,0.012165,"{'max_depth': None, 'min_samples_leaf': 0.05, ...",0.534101,0.031037
10,5,0.660499,0.111307,6,0.651099,0.111444,8,0.679448,0.19137,12,-0.122967,0.028876,"{'max_depth': None, 'min_samples_leaf': 0.01, ...",1.026682,0.075061
5,6,0.659531,0.091021,5,0.652494,0.09042,7,0.680547,0.137951,6,-0.11517,0.010628,"{'max_depth': None, 'min_samples_leaf': 0.02, ...",0.077995,0.042738
4,7,0.658274,0.083578,7,0.649863,0.080968,5,0.687135,0.143016,10,-0.118254,0.016664,"{'max_depth': None, 'min_samples_leaf': 0.02, ...",0.798387,0.087022
6,8,0.649311,0.077607,8,0.63892,0.07203,6,0.686307,0.138536,11,-0.11856,0.021699,"{'max_depth': None, 'min_samples_leaf': 0.02, ...",0.837493,0.079296
9,9,0.636623,0.109693,9,0.62974,0.100876,11,0.65415,0.112411,7,-0.116309,0.008674,"{'max_depth': None, 'min_samples_leaf': 0.01, ...",0.089184,0.062927
1,10,0.636108,0.112148,11,0.626085,0.104453,9,0.658907,0.132478,4,-0.114093,0.010116,"{'max_depth': None, 'min_samples_leaf': 0.05, ...",0.067662,0.061731
